In [1]:
import requests
import json

# If You like to test, can put url of data app and any entry app
url = '10.168.20.20:5010'
entries = ['5b68a0ecea0637002f6a4e34', '5b97f3583cdad83c6c25aa56']
rolesa = '5b688de7ea0637002f6a4e30'

#url = 'localhost:5010'
#entries = ['5b340835b3bad96ea3a82eff']
#rolesa = '593f49be42d3ed97390f25a7'

In [2]:
pipeline = [
    {'$match': {'_id': {'$in': entries}, 'roles._id': rolesa}},
    {
        '$graphLookup': {
            'from': 'applications',
            'startWith': '$deps._id',
            'connectFromField': 'deps._id',
            'connectToField': '_id',
            'as': 'nodes',
            'maxDepth': 10,
            'depthField': 'steps'
        }
    },
    {'$project': {
                    'name': 1, 'family': 1, 
                    'servers': 1,
                    'datacenters': 1,
                    'deps._id': 1, 'deps.endpoint': 1, 
                    'nodes._id': 1, 'nodes.deps': 1, 
                    'nodes.steps': 1, 'nodes.name': 1, 
                    'nodes.family': 1, 'nodes.servers': 1,
                    'nodes.provider': 1, 'nodes.datacenters': 1
                 }
    }
];

jpipeline = json.dumps(pipeline)

data = requests.post('http://%s/aggregate' % url, json={'entity': 'applications', 'pipeline': jpipeline})
data = data.json()

In [3]:
import sys
import os
import numpy as np
cwd = os.getcwd()
spltted = cwd.split('/')[:-1]
sys.path.append('/'.join(spltted))

import networkx as nx
from app.services.gridOrchestrator import GridOrchestrator
from app.libs.network.baseNetwork import BaseNetwork
from app.libs.histograms.histogram import Histogram
from app.libs.histograms.grid import GridHistogram

from app.libs.transformDict import append_servers, transform_dict

from svgwrite import Drawing

network = BaseNetwork()
network.make(data.get('items')).get_graph()

In [4]:
mapping_style = {
    'aws': '.aws{color: #f58536;fill: #c4692c;}',
    'premise': '.premise{color: #5b5b5b;fill: #8e8e8e;}',
    'openstack': '.openstack{color: #ce081f;fill: #af071a;}',
    'azure': '.azure{color: #1566bf;fill: #7396bc;}',
    'google cloudengine': '.cloudengine{color: #1566bf;fill: #3b7ec6;}',
    'digital ocean': '.ocean{color: #0080FF;fill: #046cce;}',
    'linode': '.linode{color: #08a830;fill: #037720;}',
    'rackspace': '.rackspace{color: #8e0416;fill: #c1001b;}',
    'heroku': '.heroku{color: #473C8B;fill: #6053b5;}',
    'ovh': '.ovh{color: #0B4571;fill: #155d91;}',
    'godaddy': '.godaddy{color: #02C54A;fill: #18db5f;}',
    'default': '.dark{fill: currentColor;}.darker{filter: url(#brightness)}.2darker{filter: url(#darker)}text{font-family:Arial, Helvetica, sans-serif; fill: #672064;}.white{fill:#fff;}.gray{fill:#d1d1d1;}.gdarker{fill:#bababa}.black{fill:#000}.lgray{fill:#ccc},.gray2{fill:#6d6d6d}.dark-gray2{fill:#757575}.little-white{fill:#efefef}.light-gray{fill:#f2f2f2;}'
}


class LoadTemplates(object):
    
    def __init__(self, path):
        self._path = path
        self._mapp = {}
        
        self.crawler()
    
    def __call__(self):
        return self._mapp
        
    def crawler(self):
        for (dirpath, dirnames, filenames) in os.walk(self._path):

            if filenames:
                spl = dirpath.split('/')[-1]
                self.iter_files(spl, filenames)

    
    def without_prefix(self, label):
        spt = label.split('.')[:-1]
        return ''.join(spt)
    
    def get_prefix(self, pfx):
        if pfx:
            return '%s.' % pfx
        
        return ''
    def iter_files(self, prefix, filenames):
        
        for file in filenames:
            
            name = '%s%s/%s' % (self._path, prefix, file)
            key= '%s%s' % (self.get_prefix(prefix), self.without_prefix(file))
            with open(name) as f:
                data = json.load(f)    
                self._mapp[key] = data
        
        

mypath = '/'.join(spltted) + '/assets/symbol/'

mapping = LoadTemplates(mypath)()

In [ ]:
class DrawTemplateSVG(object):
    def __init__(self, tmax, servers, grid, darea=DrawArea):
        pass
        
    def draw_app(self, item):
        pass
    

    def add(self, symbol):
        self.dwg.add(symbol)

    def save(self):
        self.dwg.save()
        return self.dwg.tostring()

    def draw_connect(self, node1, node2, details={}):


In [ ]:
class DrawLayout(object):
    def __init__(self, grid, index, servers={}, draw=DrawTemplateSVG):
        
        self._grid = grid
        self._index = index
        
        self._max_x = max(self._grid, key=int)
        self._max_y = max(self._grid[0], key=int)
        self.setup_drawer(draw, servers)
        

    def setup_drawer(self, draw, servers):
        tmax = (self._max_x, self._max_y)
        self.drawer = draw(tmax, servers, self._grid)
        
    def draw_nodes(self):
        for _, node in self._index.items():
            self.drawer.draw_app(node)
                
        return self
        
    def draw_connections(self, edges):
        for edge in edges:
            edg = [self._index[edge[x]] for x in range(2)]
            
            self.drawer.draw_connect(*edg, edge[2])
            
        return self
    
    def save(self):
        return self.drawer.save()

In [ ]:
from IPython.display import SVG, display
from functools import reduce
from app.repository.externalMaestroData import ExternalMaestroData

print("-------------------------")
Orchestration = GridOrchestrator(network.graph)
Orchestration.create(entries)


och = Orchestration.get_mapping()

servers_id = reduce(append_servers, och[1].values(), [])
servers_id = list(set(servers_id)) #remove duplicate
query = {"_id": servers_id}
ExternalRequest = ExternalMaestroData(owner_id=rolesa, graph_id="asda")
result = ExternalRequest.get_request(path="servers", query=query)

servers = transform_dict(result)

Layout = DrawLayout(*och, servers)

Layout.draw_connections(network.graph.edges(data='endpoint'))
Layout.draw_nodes()

Layout.save()